# 데이터

In [1]:
# 라이브러리 불러오기
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

import re
from collections import Counter
import sentencepiece as spm
from konlpy.tag import Okt
from konlpy.tag import Mecab
import csv 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
# 한글 폰트에 문제가 생겼을 때

# 한글 폰트 설치
!apt-get update -qq
!apt-get install -qq fonts-nanum

# 설치한 폰트를 matplotlib에서 사용할 수 있도록 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# 나눔 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 폰트 매니저에 폰트 추가
fm.fontManager.addfont(font_path)
plt.rc('font', family='NanumGothic')  # 폰트 설정

## 데이터 업로드

#### 학습 데이터 불러오기

프롬프트로 생성된 '일반 대화' 합성 데이터와 원본 데이터가 합쳐진 파일

In [ ]:
train_data_path ="../data/conversations.csv"
train_data = pd.read_csv(train_data_path)
# 원본 데이터 저장
origin_data = train_data

In [ ]:
train_data.sample(10)

In [ ]:
train_data.info()

## Augmentation

## EDA

In [ ]:
# 클래스 별 대화 확인해보기
class_counts = train_data.groupby('class').size()
class_counts

In [ ]:
# 대화(X)와 레이블(y)로 나누기
X_train = train_data['conversation']
y_train = train_data['class']

#### 길이 분포 확인

In [ ]:
# 길이 분포 확인하기
print('Data Information')
print('-'*50)
print('Conversations :',X_train.shape)
print('Label :',y_train.shape)
print('Label Type:', set(y_train))
print('-'*50)
num_classes = 4
print('The number of classes : {}'.format(num_classes))
print('-'*50)
print('Max Length :{}'.format(max(len(l) for l in X_train)))
print('Average Length :{}'.format(sum(map(len, X_train))/len(X_train)))

#### 클래스 별 분포 확인

In [ ]:
# 두 서브플롯 생성하기
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

# 샘플 길이의 히스토그램
axes[0].hist([len(s) for s in X_train], bins=50, color='skyblue')
axes[0].set_xlabel('Length of samples')
axes[0].set_ylabel('Number of samples')
axes[0].set_title('Sample Length Distribution')

# 클래스별 샘플 수 플롯
sns.countplot(x=y_train, ax=axes[1], palette='pastel')
axes[1].set_title('Class Distribution')

# 레이아웃 조정하기
plt.tight_layout()

# 플롯 표시하기
plt.show()

#### 클래스 별 길이 분포 확인

In [ ]:
# 서브플롯 생성하기
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

# 샘플 길이의 히스토그램 그리기

# 협박 대화
axes[0][0].hist([len(s) for s in train_data['conversation'][train_data['class']=='협박 대화']],
                bins=50, color='skyblue')
axes[0][0].set_xlabel('Length of samples')
axes[0][0].set_ylabel('Number of samples')
axes[0][0].set_title('협박 대화')

# 갈취 대화
axes[0][1].hist([len(s) for s in train_data['conversation'][train_data['class']=='갈취 대화']],
                bins=50, color='salmon')
axes[0][1].set_xlabel('Length of samples')
axes[0][1].set_ylabel('Number of samples')
axes[0][1].set_title('갈취 대화')

# 직장 내 괴롭힘 대화
axes[1][0].hist([len(s) for s in train_data['conversation'][train_data['class']=='직장 내 괴롭힘 대화']],
                bins=50, color='lightgreen')
axes[1][0].set_xlabel('Length of samples')
axes[1][0].set_ylabel('Number of samples')
axes[1][0].set_title('직장 내 괴롭힘 대화')

# 기타 괴롭힘 대화
axes[1][1].hist([len(s) for s in train_data['conversation'][train_data['class']=='기타 괴롭힘 대화']],
                bins=50, color='orchid')
axes[1][1].set_xlabel('Length of samples')
axes[1][1].set_ylabel('Number of samples')
axes[1][1].set_title('기타 괴롭힘 대화')

plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.show()

#### 화자 별 특징 분석

In [ ]:
# A와 B의 대화를 담을 리스트 초기화하기
A_conversations = []
B_conversations = []

In [ ]:
# 대화 열을 반복하면서 각 대화를 분할하기
for conversation in train_data['conversation']:
    try:
        lines = conversation.split('\n')
        # 홀수 줄은 A, 짝수 줄은 B에 추가하기
        A_conv = '\n'.join([line for idx, line in enumerate(lines) if idx % 2 == 0])
        B_conv = '\n'.join([line for idx, line in enumerate(lines) if idx % 2 == 1])
        A_conversations.append(A_conv)
        B_conversations.append(B_conv)
    except ValueError:
        # 대화가 정확히 두 부분으로 나뉘지 않는 경우 처리하기
        A_conversations.append(conversation)
        B_conversations.append('')

In [ ]:
# A와 B를 위한 새로운 데이터프레임 생성하기
A_data = pd.DataFrame({'idx': train_data['idx'], 'class': train_data['class'], 'conversation': A_conversations})
B_data = pd.DataFrame({'idx': train_data['idx'], 'class': train_data['class'], 'conversation': B_conversations})

In [ ]:
# A의 head() 출력하기
print("A 데이터셋의 첫 5개 행:")
A_data.head()

In [ ]:
# B의 head() 출력하기
print("B 데이터셋의 첫 5개 행:")
B_data.head()

In [ ]:
# Okt 토크나이저 활용하여 단어 추출
okt=Okt()

In [ ]:
# 가장 많이 나오는 단어 추출하는 함수
def get_top_words(dataframe, class_name, top_n=10):
    # 특정 클래스의 대화만 추출하기
    class_data = dataframe[dataframe['class'] == class_name]['conversation']
    words = []

    for conversation in class_data:
        words.extend([word for word in okt.nouns(conversation)])

    # 가장 많이 등장하는 단어 top_n 추출하기
    counter = Counter(words)
    common_words = counter.most_common(top_n)
    return common_words

In [ ]:
# 가장 많이 나오는 단어 시각화하는 함수
def plot_top_words(common_words_A, common_words_B, class_name):
    words_A, counts_A = zip(*common_words_A)
    words_B, counts_B = zip(*common_words_B)

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 8))

    axes[0].bar(words_A, counts_A, color=plt.cm.Paired.colors)
    axes[0].set_title(f'Top {len(words_A)} Words in Class {class_name} (A)')
    axes[0].set_xlabel('Words')
    axes[0].set_ylabel('Counts')

    axes[1].bar(words_B, counts_B, color=plt.cm.Paired.colors)
    axes[1].set_title(f'Top {len(words_B)} Words in Class {class_name} (B)')
    axes[1].set_xlabel('Words')
    axes[1].set_ylabel('Counts')

    plt.show()

In [ ]:
# 클래스 목록 추출하기
classes = train_data['class'].unique()

In [ ]:
# 각 클래스별로 시각화
for class_name in classes:
    print(f"Class: {class_name}")
    print("A 데이터셋의 상위 10개 단어:")
    top_words_A = get_top_words(A_data, class_name)
    print(top_words_A)

    print("\nB 데이터셋의 상위 10개 단어:")
    top_words_B = get_top_words(B_data, class_name)
    print(top_words_B)

    plot_top_words(top_words_A, top_words_B, class_name)

## 전처리

### 기본 전처리

#### 중복값 여부 찾아보기

In [ ]:
# 중복값 찾기
duplicates = train_data[train_data.duplicated()]
duplicates

중복값 없음

#### 결측치 여부 확인하기

In [ ]:
# 결측치 여부 확인하기
train_data.isnull().sum()

결측치 없음

#### 클래스 컬럼 인코딩하기

In [ ]:
# 'class'를 'type'으로 매핑하는 딕셔너리 생성하기
class_to_type = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

In [ ]:
# 'class' 열을 기반으로 새로운 'type' 열 추가하기
train_data['type'] = train_data['class'].map(class_to_type)

In [ ]:
# type 열 추가했는지 확인하기
train_data.head()

In [ ]:
# 기존 idx, class 컬럼 삭제하기
new_train_data = train_data.drop(['idx', 'class'], axis=1)
# idx, class 컬럼 삭제했는지 확인하기
new_train_data.head()

In [ ]:
# new_train_data를 train_data에 덮어 씌우기
train_data = new_train_data

### 텍스트 전처리

#### 한글 외 문자 삭제
한글, '?', '!', '.', '.', 공백 유지

In [ ]:
# 전처리 함수
def preprocess_sentence(sentence): 
    # \n을 공백으로 바꾸기
    sentence = re.sub("\n", " ", sentence)
    
    # (ㄱ-ㅎ, ㅏ-ㅣ, ".", "?", "!", ",", ' ')를 제외한 모든 문자를 없애기
    sentence = re.sub("[^ㄱ-ㅣ가-힣.?!, ]", "", sentence)
    
    # 단어와 구두점(punctuation) 사이에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    
    return sentence

In [ ]:
# 전처리 데이터 새로운 column에 저장
train_data['preprocessed'] = train_data['conversation'].apply(preprocess_sentence)
train_data['preprocessed']

In [ ]:
# 기존 'conversation' column 전처리한 데이터로 바꾸기
train_data['conversation'] = train_data['preprocessed']
train_data.drop('preprocessed', axis=1)

#### 불용어 삭제
[불용어 리스트 출처](https://www.ranks.nl/stopwords/korean)

In [ ]:
# 불용어 리스트 불러오기
stopwords_path = "ko_stopwords.txt"
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

#### 토큰화

```!pip install sentencepiece```  
```!pip install konlpy```

In [ ]:
# 단어사전 크기 20000으로 제한
VOCAB_SIZE=20000

토크나이저 선정
1. SentencePiece  
[ref](https://wikidocs.net/86657)

In [ ]:
# sentencePiece의 입력으로 사용하기 위해서 데이터를 txt 파일로 저장
with open('conversations.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_data['conversation']))

spm.SentencePieceTrainer.Train(
    '--input=conversations.txt ' + 
    '--model_prefix=dktc ' +
    f'--vocab_size={VOCAB_SIZE} ' +
    '--model_type=bpe ' + 
    '--pad_id=0 --pad_piece=<pad> ' +
    '--unk_id=1 --unk_piece=<unk> ' +
    '--bos_id=2 --bos_piece=<sos> ' +
    '--eos_id=3 --eos_piece=<eos> ' + 
    '--max_sentence_length=9999'
)

In [ ]:
word_index_df = pd.read_csv('dktc.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
word_index_df.columns=['words', 'idx']
word_index_df['idx'] = np.arrange(len(word_index_df))
word_to_index = word_index_df.to_dict()

In [ ]:
word_to_index

2. konlpy  
1) Okt  
2) Mecab

In [ ]:
# Okt로 토큰화
tokenizer=Okt()
#tokenizer=Mecab()

def tokenize(conversation, tokenizer):
    return [token for token in tokenizer.morphs(conversation) if token not in stopwords]

# 각 conversation을 토큰화하여 새로운 열 'tokenized'에 저장
train_data['tokenized'] = train_data['conversation'].apply(lambda x: tokenize(x, tokenizer))

#### 시작 토큰, 종료 토큰 추가

In [ ]:
special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']

# 시작 토큰과 종료 토큰을 추가하는 함수 정의
def add_special_tokens(tokens):
    return [special_tokens[2]] + tokens + [special_tokens[3]]

# 'tokenized' 열에 함수 적용하여 시작 토큰과 종료 토큰 추가
train_data['tokenized'] = train_data['tokenized'].apply(add_special_tokens)

In [ ]:
# 토큰화된 단어 확인
train_data['tokenized']

#### 단어사전 생성

In [ ]:
# 'tokenized' 열의 모든 토큰을 모아 등장 빈도 계산
all_tokens = [token for tokens in train_data['tokenized'] for token in tokens]
counter = Counter(all_tokens)
# 가장 많이 등장하는 단어 vocab_size 개수만큼 남기기
counter = counter.most_common(VOCAB_SIZE)

In [ ]:
# vocabulary 파일로 저장
word_to_index_path = './vocab.txt'

def save_word_index(counter):
    with open(word_to_index_path, 'w') as f:
        for idx, (word, _) in enumerate(counter):
            line = f"{word}: {idx}\n"
            f.write(line)
            
#save_word_index(counter)

In [ ]:
word_to_index = {token: idx for idx, token in enumerate(special_tokens[:2])}
word_to_index.update({token: idx + len(special_tokens[:2]) for idx, (token, _) in enumerate(counter)})

In [ ]:
word_to_index

#### 정수인코딩

In [ ]:
# 'tokenized' 열의 데이터를 정수 인코딩
def encode_tokens(tokens, word_to_index):
    unk_index = word_to_index['<unk>']
    return [word_to_index.get(token, unk_index) for token in tokens]

train_data['encoded'] = train_data['tokenized'].apply(lambda x: encode_tokens(x, word_to_index))

In [ ]:
train_data.sample(3)

## 전처리 후 분석

#### 토큰화된 데이터 길이 확인

In [ ]:
# 'encoded' 열의 각 리스트의 길이 계산
lengths = train_data['encoded'].apply(len)

print(f"최대 길이: {lengths.max()}")
print(f"최소 길이: {lengths.min()}")
print(f"평균 길이: {lengths.mean()}")
print(f"표준편차: {lengths.std()}")

# 길이 분포 시각화
plt.hist(lengths, bins=50, color='skyblue')
plt.show()

#### 패딩 적용

In [ ]:
# 대화 최대 길이 150으로 설정
MAX_LENGTH = 150

In [ ]:
train_data['encoded'] = pad_sequences(train_data['encoded'], maxlen=MAX_LENGTH, padding='post', truncating='post')

## 데이터 분할

In [ ]:
X = train_data['encoded']
y = train_data['type']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, shuffle=True, random_state=42)

print('훈련 데이터의 개수 :', len(X_train))
print('훈련 레이블의 개수 :', len(y_train))
print('검증 데이터의 개수 :', len(X_val))
print('검증 레이블의 개수 :', len(y_val))
print('테스트 데이터의 개수 :', len(X_test))
print('테스트 레이블의 개수 :', len(y_test))

# 모델링

In [ ]:
!pip install wandb==0.16.0

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login(key = '809618c39f10bc0019fd6fd710cb28c698c30197')

In [ ]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

In [ ]:
def train():
    default_config = {
        "vocab" : 1000,
        "embeddings" : 64,
        "units_1" : 256,
        "units_2" : 256,
        "units_3" : 1024,
        "class_num" : 4,
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 5,
        "batch_size" : 32
    }

    wandb.init(config = default_config)
    config = wandb.config

    # Model

    model=keras.models.Sequential()
    model.add(keras.layers.Embedding(config.vocab, config.embeddings))
    model.add(keras.layers.GRU(units = config.units_1, return_sequences = True))
    model.add(keras.layers.GRU(units = config.units_2))
    model.add(keras.layers.Dense(config.units_3, activation='relu'))
    model.add(keras.layers.Dense(config.class_num, activation='softmax'))

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.
    
    model.fit(X_train, y_train,
              epochs = config.epoch,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback()])
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.
    
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})

In [ ]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = '4-rldur0',
                       project = 'first-wandb-project')

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)